# **How to Get insightful information about a Stock performance in one click with code interpreter agent using LlamaIndex and Anthropic**




*  The goal of this agent is to request from an LLM to propose a python code to fetch historical prices for a given stock on the current month, and get analysis based on these prices.

*  I built this agent using FunctionCallingAgent abstraction from LlamaIndex, and Code Interpreter tool.

*  Trick: I asked first for a code to detect which date is today, and then instruct it to fetch historical prices based on this date. If you don't do that, It will take


In [10]:
%pip install llama-index-llms-anthropic -q
%pip install llama-index-tools-code-interpreter -q
%pip install llama-index yfinance -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

CLAUDE_API_KEY = os.getenv('ANTHROPIC_API_KEY')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [3]:
from llama_index.tools.code_interpreter.base import CodeInterpreterToolSpec

In [4]:
code_spec = CodeInterpreterToolSpec()

tools = code_spec.to_tool_list()

# Specify the Agent:

Using FunctionCallingAgent to build our agent. This object allows wokring with an LLM that supports function calling like Anthropic.

In the tools, we'll specify the code interpreter tool, which is a LLamaIndex package.

In [5]:
from llama_index.llms.anthropic import Anthropic
from llama_index.core import Settings

tokenizer = Anthropic().tokenizer
Settings.tokenizer = tokenizer

llm_claude = Anthropic(model="claude-3-7-sonnet-20250219")

from llama_index.core.agent import FunctionCallingAgent

agent = FunctionCallingAgent.from_tools(
    tools,
    llm=llm_claude,
    verbose=True,
    allow_parallel_tool_calls=False,
)

In [6]:
stock = 'NVDA'

prompt = f"""
Write a python code to :
- Detect which date is today
- Based on this date, fetch historical prices of {stock} from the beginning of the month until today.
- Analyze the last month prices
"""

resp = agent.chat(prompt)

> Running step 9eead55a-663f-47ed-9b9a-83b929086885. Step input: 
Write a python code to :
- Detect which date is today
- Based on this date, fetch historical prices of NVDA from the beginning of the month until today.
- Analyze the last month prices

Added user message to memory: 
Write a python code to :
- Detect which date is today
- Based on this date, fetch historical prices of NVDA from the beginning of the month until today.
- Analyze the last month prices

=== LLM Response ===
I'll write a Python code to detect today's date, fetch NVIDIA (NVDA) stock prices from the beginning of the current month until today, and analyze the last month's prices.

Let me create this for you:
=== Calling Function ===
Calling function: code_interpreter with args: {}
=== Function Output ===
Encountered error: CodeInterpreterToolSpec.code_interpreter() missing 1 required positional argument: 'code'
> Running step 9d975f58-b02c-4931-a8ef-d76f958c479b. Step input: None
=== LLM Response ===
I need to p

In [7]:
print(resp.response)

I apologize for the technical issues. Let me try again with the proper code parameter:


Access to the Agent Memory

In [8]:
agent.memory

ChatMemoryBuffer(chat_store=SimpleChatStore(store={'chat_history': [ChatMessage(role=<MessageRole.USER: 'user'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='\nWrite a python code to :\n- Detect which date is today\n- Based on this date, fetch historical prices of NVDA from the beginning of the month until today.\n- Analyze the last month prices\n')]), ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={'tool_calls': [{'id': 'toolu_016gaahqsXTcDZSDgREgBxAX', 'input': {}, 'name': 'code_interpreter', 'type': 'tool_use'}]}, blocks=[TextBlock(block_type='text', text="I'll write a Python code to detect today's date, fetch NVIDIA (NVDA) stock prices from the beginning of the current month until today, and analyze the last month's prices.\n\nLet me create this for you:")]), ChatMessage(role=<MessageRole.TOOL: 'tool'>, additional_kwargs={'name': 'code_interpreter', 'tool_call_id': 'toolu_016gaahqsXTcDZSDgREgBxAX'}, blocks=[TextBlock(block_type='text', 

In [1]:
import yfinance as yf
import pandas as pd
from datetime import datetime, date

# Get today's date
today = date.today()
print(f"Today's date is: {today}")

# Get the first day of the current month
start_date = today.replace(day=1)
print(f"Start date is: {start_date}")

# Fetch Tesla stock data
tesla = yf.download('TSLA', start=start_date, end=today)
print("\nTesla stock data shape:", tesla.shape)

# Basic analysis of the stock prices
print("\nPrice Summary Statistics:")
print(tesla['Close'].describe())

# Calculate daily returns
tesla['Daily_Return'] = tesla['Close'].pct_change()

# Print key metrics
print("\nKey Metrics:")
print(f"Starting Price: ${float(tesla['Close'].iloc[0]):.2f}")
print(f"Ending Price: ${float(tesla['Close'].iloc[-1]):.2f}")
print(f"Highest Price: ${float(tesla['High'].max()):.2f}")
print(f"Lowest Price: ${float(tesla['Low'].min()):.2f}")
print(f"Average Daily Return: {float(tesla['Daily_Return'].mean()*100):.2f}%")
print(f"Daily Return Volatility: {float(tesla['Daily_Return'].std()*100):.2f}%")

# Calculate total return for the period
total_return = float(((tesla['Close'].iloc[-1] - tesla['Close'].iloc[0]) / tesla['Close'].iloc[0]) * 100)
print(f"\nTotal Return for the period: {total_return:.2f}%")

Today's date is: 2025-03-05
Start date is: 2025-03-01
YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


Tesla stock data shape: (2, 5)

Price Summary Statistics:
Ticker        TSLA
count     2.000000
mean    278.345001
std       8.916606
min     272.040009
25%     275.192505
50%     278.345001
75%     281.497498
max     284.649994

Key Metrics:
Starting Price: $284.65
Ending Price: $272.04
Highest Price: $303.94
Lowest Price: $261.84
Average Daily Return: -4.43%
Daily Return Volatility: nan%

Total Return for the period: -4.43%



/var/folders/pn/hshghr557y1c87_6431qwc3m0000gn/T/ipykernel_3998/1533891465.py:26: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  print(f"Starting Price: ${float(tesla['Close'].iloc[0]):.2f}")
/var/folders/pn/hshghr557y1c87_6431qwc3m0000gn/T/ipykernel_3998/1533891465.py:27: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  print(f"Ending Price: ${float(tesla['Close'].iloc[-1]):.2f}")
/var/folders/pn/hshghr557y1c87_6431qwc3m0000gn/T/ipykernel_3998/1533891465.py:28: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  print(f"Highest Price: ${float(tesla['High'].max()):.2f}")
/var/folders/pn/hshghr557y1c87_6431qwc3m0000gn/T/ipykernel_3998/1533891465.py:29: FutureWarning: Calling float on a single element Ser